<a href="https://colab.research.google.com/github/FitriRamadhania/Tugas3_klasifikasi_gambar_multilabe/blob/main/Tugas3_DeepL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.layers import BatchNormalization

In [3]:
#image_directory = 'DatasetMultilabe_Fitri Ramadhania_202255202021l/images/'

#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("fitriramadhania/dataset-sunscreen")

#print("Path to dataset files:", path)

In [4]:
import os
os.environ["KAGGLE_USERNAME"] = "fitriramadhania" # Ganti dengan username Kaggle Anda
os.environ["KAGGLE_KEY"] = "d1c74f5a85cd21bc1f8ababb8f7d8951" # Ganti dengan key yang ada di kaggle.json

In [5]:
!kaggle datasets download -d fitriramadhania/dataset-sunscreen

Dataset URL: https://www.kaggle.com/datasets/fitriramadhania/dataset-sunscreen
License(s): other
 97% 648M/671M [00:05<00:00, 139MB/s]
100% 671M/671M [00:05<00:00, 128MB/s]


In [6]:
# prompt: unzip file dataset-sunscreen.zip

!unzip dataset-sunscreen.zip


Archive:  dataset-sunscreen.zip
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar01.jpg  
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar02.jpg  
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar03.jpg  
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar04.jpg  
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar05.jpg  
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar06.jpg  
  inflating: DatasetMultilabe_Fitri Ramadhania_202255202021-20250607T051621Z-1-001/DatasetMultilab

In [7]:
#Now let us read metadata to get our Y values (multiple lables)
# Try reading with semicolon as delimiter
try:
    df = pd.read_csv('//content/TrainDataset_Fitri Ramadhania_202255202021.csv', sep=';')
    print("Read with semicolon delimiter successful.")
except pd.errors.ParserError: # Changed to pd.errors.ParserError for clarity
    print("Reading with semicolon delimiter failed. Trying default comma delimiter (might still fail).")
    df = pd.read_csv('/content/TrainDataset_Fitri Ramadhania_202255202021.csv')

print(df.head())     # printing first five rows of the file
print(df.columns)

df = df.iloc[:2000].reset_index(drop=True)  #Loading only first 2000 datapoints and reset the index
#Need to read images using the tag from metadata.
#Otherwise, if read directly from the folder then images may not correspond to
#the metadata from the csv file.

Read with semicolon delimiter successful.
         Id                                    jenis sunscreen  \
0  gambar01  tinted sunscreen, sunscreen spray, sunscreen g...   
1  gambar02  tinted sunscreen, sunscreen spray, sunscreen g...   
2  gambar03  tinted sunscreen, sunscreen spray, sunscreen g...   
3  gambar04  tinted sunscreen, sunscreen spray, sunscreen g...   
4  gambar05  tinted sunscreen, sunscreen spray, sunscreen g...   

   tinted sunscreen  sunscreen spray  sunscreen gel  body sunscreen  
0                 1                1              1               1  
1                 1                1              1               1  
2                 1                1              1               1  
3                 1                1              1               1  
4                 1                1              1               1  
Index(['Id', 'jenis sunscreen', 'tinted sunscreen', 'sunscreen spray',
       'sunscreen gel', 'body sunscreen'],
      dtype='object')


In [8]:
!ls /content/sample_data/

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [9]:
!ls {image_directory} | head -n 20

ls: cannot access '{image_directory}': No such file or directory


In [10]:
print(df.columns)


Index(['Id', 'jenis sunscreen', 'tinted sunscreen', 'sunscreen spray',
       'sunscreen gel', 'body sunscreen'],
      dtype='object')


In [11]:
print([col for col in df.columns if 'id' in col.lower()])


['Id']


In [12]:

#print(df['Id'][500])  #This line was causing an error if index 500 does not exist
print(df['Id'][df.shape[0] - 1]) # Accessing the 'id' of the last row

gambar300


In [13]:
print(df['Id'].head())


0    gambar01
1    gambar02
2    gambar03
3    gambar04
4    gambar05
Name: Id, dtype: object


In [14]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
from tqdm import tqdm

SIZE = 200
X_dataset = []

# Direktori gambar
image_directory = '/content/DatasetMultilabe_Fitri Ramadhania_202255202021/'

# Looping untuk membaca gambar
for i in tqdm(range(df.shape[0])):
    image_id = str(df['Id'].iloc[i])  # pastikan tipe string
    img_path = os.path.join(image_directory, image_id + '.jpg')

    try:
        img = image.load_img(img_path, target_size=(SIZE, SIZE))
        img = image.img_to_array(img)
        img = img / 255.0
        X_dataset.append(img)
    except FileNotFoundError:
        print(f"[WARNING] Gambar tidak ditemukan: {img_path}")
        continue  # lanjut ke data berikutnya jika gambar hilang

# Konversi jadi array NumPy
X = np.array(X_dataset)


100%|██████████| 300/300 [00:00<00:00, 27769.49it/s]

[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar01.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar02.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar03.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar04.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar05.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar06.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar07.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar08.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar09.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadha

In [15]:

#print(df['id'][150])  #This line was causing an error if index 1500 does not exist
print(df['Id'][df.shape[0] - 1]) # Accessing the 'id' of the last row

gambar300


In [16]:
#Now let us read metadata to get our Y values (multiple lables)
# Specify the delimiter as semicolon based on the column names observed
df = pd.read_csv('TrainDataset_Fitri Ramadhania_202255202021.csv', sep=';')
print(df.head())     # printing first five rows of the file
print(df.columns)

df = df.iloc[:2000]  #Loading only first 2000 datapoints for memory reasons
#Need to read images using the tag from metadata.
#Otherwise, if read directly from the folder then images may not correspond to
#the metadata from the csv file.

         Id                                    jenis sunscreen  \
0  gambar01  tinted sunscreen, sunscreen spray, sunscreen g...   
1  gambar02  tinted sunscreen, sunscreen spray, sunscreen g...   
2  gambar03  tinted sunscreen, sunscreen spray, sunscreen g...   
3  gambar04  tinted sunscreen, sunscreen spray, sunscreen g...   
4  gambar05  tinted sunscreen, sunscreen spray, sunscreen g...   

   tinted sunscreen  sunscreen spray  sunscreen gel  body sunscreen  
0                 1                1              1               1  
1                 1                1              1               1  
2                 1                1              1               1  
3                 1                1              1               1  
4                 1                1              1               1  
Index(['Id', 'jenis sunscreen', 'tinted sunscreen', 'sunscreen spray',
       'sunscreen gel', 'body sunscreen'],
      dtype='object')


In [17]:
#
print(df['Id'][17])  #tinted sunscreen, sunscreen spray, sunscreen gel, body sunscreen

gambar18


In [18]:
print(df['jenis sunscreen'][4])  #Tagged as multiple Genres.

tinted sunscreen, sunscreen spray, sunscreen gel, body sunscreen


In [19]:
from sklearn.model_selection import train_test_split
import numpy as np

# Pastikan kolom 'Id' dan 'jenis sunscreen' ada di DataFrame sebelum di-drop
drop_cols = ['Id', 'jenis sunscreen']
for col in drop_cols:
    if col not in df.columns:
        raise ValueError(f"Kolom '{col}' tidak ditemukan di DataFrame!")

In [20]:
# Ambil label multilabel (yang akan diprediksi)
y = df.drop(columns=drop_cols).values  # bentuknya array 2D

In [21]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))

#Do not use softmax for multilabel classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins.

#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#like multi label, in this example.
#But, also good for binary mutually exclusive (cat or not cat).

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 196, 196, 16)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 196, 196, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 98, 98, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 98, 98, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 94, 94, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 47, 47, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 43, 43, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 17, 17, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 702,777 (2.68 MB)

 Trainable params: 702,425 (2.68 MB)

 Non-trainable params: 352 (1.38 KB)

In [22]:
#Binary cross entropy of each label. So no really a binary classification problem but
#Calculating binary cross entropy for each label.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [23]:
# Direktori gambar
image_directory = '/content/DatasetMultilabe_Fitri Ramadhania_202255202021/'


In [24]:
# Looping untuk membaca gambar
for i in tqdm(range(df.shape[0])):
    image_id = str(df['Id'].iloc[i])  # pastikan tipe string
    img_path = os.path.join(image_directory, image_id + '.jpg')

    try:
        img = image.load_img(img_path, target_size=(SIZE, SIZE))
        img = image.img_to_array(img)
        img = img / 255.0
        X_dataset.append(img)
    except FileNotFoundError:
        print(f"[WARNING] Gambar tidak ditemukan: {img_path}")
        continue  # lanjut ke data berikutnya jika gambar hilang


100%|██████████| 300/300 [00:00<00:00, 18059.18it/s]

[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar01.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar02.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar03.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar04.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar05.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar06.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar07.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar08.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadhania_202255202021/gambar09.jpg
[WARNING] Gambar tidak ditemukan: /content/DatasetMultilabe_Fitri Ramadha

In [26]:
# Konversi jadi array NumPy
X = np.array(X_dataset)


In [28]:
# Ambil semua label unik dari kolom 'jenis sunscreen'
all_labels = set()
for labels_str in df['jenis sunscreen']:
    labels = [label.strip() for label in labels_str.split(',')]
    all_labels.update(labels)


In [29]:
# Tampilkan jumlah label unik
num_unique_labels = len(all_labels)
print(f"Jumlah label unik yang ditemukan: {num_unique_labels}")


Jumlah label unik yang ditemukan: 4


In [30]:
# %%
from sklearn.model_selection import train_test_split
import numpy as np


In [31]:
# Pastikan kolom 'Id' dan 'jenis sunscreen' ada di DataFrame sebelum di-drop
drop_cols = ['Id', 'jenis sunscreen']
for col in drop_cols:
    if col not in df.columns:
        raise ValueError(f"Kolom '{col}' tidak ditemukan di DataFrame!")


In [32]:
# %%
# Ambil label multilabel (yang akan diprediksi)
y = df.drop(columns=drop_cols).values  # bentuknya array 2D


In [33]:
# %%
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
#Do not use softmax for multilabel classification
#Softmax is useful for mutually exclusive classes, either cat or dog but not both.
#Also, softmax outputs all add to 1. So good for multi class problems where each
#class is given a probability and all add to 1. Highest one wins.

#Sigmoid outputs probability. Can be used for non-mutually exclusive problems.
#like multi label, in this example.
#But, also good for binary mutually exclusive (cat or not cat).

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 196, 196, 16)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 196, 196, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 98, 98, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 98, 98, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 94, 94, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 47, 47, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 43, 43, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 21, 21, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 17, 17, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 25)             │         1,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 702,777 (2.68 MB)

 Trainable params: 702,425 (2.68 MB)

 Non-trainable params: 352 (1.38 KB)

In [35]:
# %%
#Binary cross entropy of each label. So no really a binary classification problem but
#Calculating binary cross entropy for each label.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [36]:
# %%
# Kompilasi model
import tensorflow as tf # Import tensorflow needed for optimizers

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',        # Untuk multilabel classification
    metrics=['accuracy']
)

In [37]:
# %%
# Ambil semua label unik dari kolom 'jenis sunscreen'
all_labels = set()
for labels_str in df['jenis sunscreen']:
    labels = [label.strip() for label in labels_str.split(',')]
    all_labels.update(labels)


In [38]:
# %%
# Tampilkan jumlah label unik
num_unique_labels = len(all_labels)
print(f"Jumlah label unik yang ditemukan: {num_unique_labels}")


Jumlah label unik yang ditemukan: 4


In [39]:
# %%
# Proses label ke format multilabel (multi-hot encoding)
from sklearn.preprocessing import MultiLabelBinarizer # Import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df['label_list'] = df['jenis sunscreen'].apply(lambda x: [label.strip() for label in x.split(',')])
y = mlb.fit_transform(df['label_list'])


In [40]:
# %%
#plot the training and validation accuracy and loss at each epoch
# history variable is not defined in this context, assuming it will be defined by model training later
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# epochs = range(1, len(loss) + 1)
# plt.plot(epochs, loss, 'y', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [41]:
# %%
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# plt.plot(epochs, acc, 'y', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

In [45]:
# %%
###################################################

# X_test and y_test are not defined in this context, assuming they will be defined by train_test_split later
# _, acc = model.evaluate(X_test, y_test)
# print("Accuracy = ", (acc * 100.0), "%")

################################################################


In [47]:
# %%
# prompt: berikan code evaluasi model

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split # Ensure train_test_split is imported if not already


In [51]:
# Proses label ke format multilabel (multi-hot encoding)
mlb = MultiLabelBinarizer()
df['label_list'] = df['jenis sunscreen'].apply(lambda x: [label.strip() for label in x.split(',')])
y = mlb.fit_transform(df['label_list'])


In [58]:
SIZE = 200  # Ukuran target gambar
image_directory = '/content/DatasetMultilabe_Fitri Ramadhania_202255202021l'
csv_path = '/content/TrainDataset_Fitri Ramadhania_202255202021.csv'  # Ubah sesuai nama file CSV kamu


In [59]:
df = pd.read_csv(csv_path)

In [65]:
#plot the training and validation accuracy and loss at each epoch
# Ensure 'history' variable is available after model training
# For example, if you train the model like this:
# history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

if 'history' in locals():
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
else:
    print("Error: 'history' variable not found. Please train the model first.")

Error: 'history' variable not found. Please train the model first.


In [67]:
# plot the training and validation accuracy
# Ensure 'history' variable is available after model training
# Ensure 'epochs' variable is available (calculated from the length of loss/accuracy)
if 'history' in locals():
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    # Re-calculate epochs just in case, though it should be the same as for loss
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'y', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
else:
    print("Error: 'history' variable not found. Please train the model first.")

Error: 'history' variable not found. Please train the model first.


In [69]:
#Validate on an image
# Update image path to be within the extracted dataset directory
# Replace 'YOUR_IMAGE_FILENAME.jpg' with an actual image filename from your dataset
# You need to find a valid image file within the extracted dataset directory.
# For example, you could try listing files in that directory to find one.
# Example: !ls /content/DatasetMultilabe_Fitri Ramadhania_202255202021/
# Once you find a filename, replace 'YOUR_IMAGE_FILENAME.jpg' with it.
img_path_to_validate = '/content/DatasetMultilabe_Fitri Ramadhania_202255202021/YOUR_IMAGE_FILENAME.jpg' # <--- Ganti dengan nama file gambar yang sebenarnya

try:
    img = image.load_img(img_path_to_validate, target_size=(SIZE, SIZE))

    img = image.img_to_array(img)
    img = img/255.
    plt.imshow(img)
    plt.show() # Tampilkan gambar
    img = np.expand_dims(img, axis=0)

    # Ensure classes are correctly derived from the binarizer after fitting
    # Assuming mlb (MultiLabelBinarizer) has been fitted earlier in the code
    if 'mlb' in globals() and hasattr(mlb, 'classes_'):
        classes = mlb.classes_
        proba = model.predict(img)  #Get probabilities for each class
        sorted_categories = np.argsort(proba[0])[:-11:-1]  #Get class names for top 10 categories

        # Print classes and corresponding probabilities
        print("Top 10 predicted categories:")
        for i in range(10):
            print("{}".format(classes[sorted_categories[i]])+" ({:.3})".format(proba[0][sorted_categories[i]]))
    else:
        print("Error: MultiLabelBinarizer (mlb) not fitted. Please run the cell to process labels first.")

except FileNotFoundError:
    print(f"Error: Image file not found at {img_path_to_validate}. Please ensure the path and filename are correct.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: Image file not found at /content/DatasetMultilabe_Fitri Ramadhania_202255202021/YOUR_IMAGE_FILENAME.jpg. Please ensure the path and filename are correct.


In [71]:
#Print classes and corresponding probabilities
# This block assumes that 'classes', 'sorted_categories', and 'proba'
# were successfully generated in the previous cell block (Validate on an image).
# It is good practice to include error handling or checks if this cell
# can be run independently.

# Check if the necessary variables exist before attempting to print
if 'classes' in locals() and 'sorted_categories' in locals() and 'proba' in locals():
    print("Top 10 predicted categories:") # Add a header for clarity
    for i in range(10):
        print("{}".format(classes[sorted_categories[i]])+" ({:.3})".format(proba[0][sorted_categories[i]]))
else:
    print("Error: Necessary variables (classes, sorted_categories, proba) not found. Please run the 'Validate on an image' cell first.")

Error: Necessary variables (classes, sorted_categories, proba) not found. Please run the 'Validate on an image' cell first.


In [74]:
###################################################

# X_test and y_test are not defined in this context, assuming they will be defined by train_test_split later
if 'X_test' in locals() and 'y_test' in locals():
    _, acc = model.evaluate(X_test, y_test, verbose=0) # Added verbose=0 to prevent printing progress bar
    print("Accuracy = ", (acc * 100.0), "%")
else:
    print("Error: X_test and y_test are not defined. Please ensure train_test_split has been run.")

################################################################

Error: X_test and y_test are not defined. Please ensure train_test_split has been run.


In [79]:
# prompt: berikan code evaluasi model

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing import image

# Define SIZE and image_directory (assuming these are set earlier in your notebook)
SIZE = 200  # Example size, adjust as needed
image_directory = '/content/DatasetMultilabe_Fitri Ramadhania_202255202021/' # Example directory, adjust as needed
csv_path = '/content/TrainDataset_Fitri Ramadhania_202255202021.csv' # Example path, adjust as needed

# --- Start: Ensure X and y are properly loaded and processed ---
# Re-load the dataframe to be absolutely sure
try:
    df = pd.read_csv(csv_path, sep=';')
except pd.errors.ParserError:
    df = pd.read_csv(csv_path)

# Limit to 2000 datapoints and reset index if that was your intention
df = df.iloc[:2000].reset_index(drop=True)

# Process images to create X
X_dataset = []
print("Loading images for evaluation...")
for i in tqdm(range(df.shape[0])):
    image_id = str(df['Id'].iloc[i])
    img_path = os.path.join(image_directory, image_id + '.jpg')

    try:
        img = image.load_img(img_path, target_size=(SIZE, SIZE))
        img = image.img_to_array(img)
        img = img / 255.0
        X_dataset.append(img)
    except FileNotFoundError:
        # This warning was already in your code, keeping it
        # print(f"[WARNING] Gambar tidak ditemukan: {img_path}")
        continue

X = np.array(X_dataset)
print(f"Finished loading {X.shape[0]} images.")


# Process labels to create y using MultiLabelBinarizer
if 'jenis sunscreen' in df.columns:
    mlb = MultiLabelBinarizer()
    df['label_list'] = df['jenis sunscreen'].apply(lambda x: [label.strip() for label in x.split(',')])
    y = mlb.fit_transform(df['label_list'])
    print(f"Finished processing labels. y shape: {y.shape}")
else:
    print("Error: 'jenis sunscreen' column not found in the DataFrame. Cannot process labels.")
    # You might want to exit or handle this error more explicitly
    # For now, setting y to None to prevent further errors if the column is missing
    y = None


# --- End: Ensure X and y are properly loaded and processed ---


# Now, perform the train-test split and evaluation only if X and y are available
if 'X' in locals() and X is not None and 'y' in locals() and y is not None:
    print("Performing train-test split and evaluation...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Convert probabilities to class labels (e.g., using a threshold)
    y_pred_classes = (y_pred > 0.5).astype(int)  # Assuming 0.5 as the threshold

    # Calculate and print the classification report
    # Ensure target_names are the actual class names from your MultiLabelBinarizer
    if 'mlb' in globals() and hasattr(mlb, 'classes_'):
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred_classes, target_names=mlb.classes_))

        # Calculate and print the confusion matrix for each label
        print("\nConfusion Matrices per Label:")
        for i in range(len(mlb.classes_)):
            print(f"\nConfusion Matrix for {mlb.classes_[i]}:")
            print(confusion_matrix(y_test[:,i], y_pred_classes[:,i]))
    else:
        print("\nError: MultiLabelBinarizer (mlb) not fitted. Cannot get class names for evaluation.")

else:
    print("\nError: X or y are not defined or are None. Please ensure data loading and preprocessing steps have been completed successfully.")

Loading images for evaluation...


100%|██████████| 300/300 [00:00<00:00, 31913.65it/s]

Finished loading 0 images.
Finished processing labels. y shape: (300, 4)
Performing train-test split and evaluation...


ValueError: Found input variables with inconsistent numbers of samples: [0, 300]